# Library Import

In [84]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings

warnings.filterwarnings(action = 'ignore')

# Make Function for Analyze

In [85]:
def make_datatime(x):
    x = str(x)
    year = int(x[:4])
    month = int(x[4:6])
    day = int(x[6:8])
    hour = int(x[8:10])

    return dt.datetime(year,month,day,hour)

def string2num(x):
    x = re.sub(r"[^0-9]+",'',str(x))
    if x == '':
        return 0
    else:
        return int(x)
#     mim = int(x[10:12])
#     sec = int(x[12:])

# Road Data

In [86]:
train_err = pd.read_csv('data/train_err_data.csv')
display(train_err.head())

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [87]:
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

In [88]:
print(np.sort(train_err.errtype.unique()))

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 30 31 32 33 34 35 36 37 38 39 40 41 42]


# Make Data set only use user_id & errtype
* count errtyp by number of occurrences to all days
* pandas group by need to a lot of resource
* so implement numpy placeholder

## Reference) Daily aggregated data generation code

```
train_err             = pd.read_csv(PATH+'train_err_data.csv')
train_err['datetime'] = train_err['time'].apply(make_datetime)

min_day = train_err.datetime.min().date()

train_err['days'] = train_err.loc[:,'datetime'].dt.date - min_day
train_err['days'] = train_err['days'].dt.days
display(train_err)

id_error = train_err[['user_id','errtype','days']].values
error = np.zeros((train_user_number,42, 33))
for idx, err, days in tqdm(id_error):
    error[idx - train_user_id_min,err - 1, days] += 1
error.shape
```

In [97]:
id_error = train_err[['user_id','errtype']].values
error = np.zeros((train_user_number,42))

for person_idx, err in tqdm(id_error):
    error[person_idx - train_user_id_min,err - 1] += 1
error.shape

100%|██████████| 16554663/16554663 [00:34<00:00, 477504.38it/s]


(15000, 42)

In [106]:
pd.DataFrame(error)

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,...,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0
2,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,...,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,...,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,...,16.0,17.0,0.0,1.0,1.0,0.0,0.0,58.0,8.0,5.0
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15.0,12.0,0.0,1.0,1.0,0.0,0.0,6.0,0.0,0.0


# problem

In [104]:
id_error

array([[10000,    15],
       [10000,    12],
       [10000,    11],
       ...,
       [24999,     4],
       [24999,     4],
       [24999,    15]])

In [149]:
train_prob = pd.read_csv('data/train_problem_data.csv')
problem = np.zeros(15000)
problem[train_prob.user_id.unique() - 10000] = 1
problem.shape

(15000,)

In [150]:
train_prob.head()

,user_id,time
0,19224,20201102200000
1,23664,20201116140000
2,15166,20201114130000
3,12590,20201108210000
4,15932,20201103210000


In [151]:
train_prob.user_id.unique()  - 10000

array([ 9224, 13664,  5166, ...,  9114, 11505,  8822])

In [153]:
pd.DataFrame(problem)

,0
0,0.0
1,1.0
2,0.0
3,0.0
4,1.0
...,...
14995,0.0
14996,0.0
14997,1.0
14998,1.0


# Model Training

In [123]:
pd.DataFrame(problem).iloc[13665]

0    0.0
Name: 13665, dtype: float64

In [124]:
train_x = error
train_y = problem
del error,problem
print(train_x.shape,'\n',train_y.shape)

(15000, 42) 
 (15000,)


## validation auc score check method

In [129]:
def f_pr_auc(probas_pred, y_true):
    labels = y_true.get_label()
    p,r,_ = precision_recall_curve(labels, probas_pred)
    score = auc(r,p)
    return "pr+auc", score, True

In [126]:
train_x

array([[  0.,   0.,   8., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ..., 113.,  56.,   1.],
       [  0.,   0.,   2., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,  58.,   8.,   5.],
       [  0.,   0.,   0., ...,   6.,   0.,   0.],
       [  0.,   0.,   4., ...,   0.,   0.,   0.]])

# set hyper parameters

In [130]:
models = []
recalls = []
precisions = []
auc_scores = []
threshold = 0.5

params = {
    'boosting_type':'gbdt',
    'objective': 'binary',
    'metric' : 'auc',
    'seed' : 1015
}

# k-fold cross validation(k = 5)

In [131]:
k_fold = KFold(n_splits = 5, shuffle = True, random_state = 42)
for train_idx, val_idx in k_fold.split(train_x):
    
    #split train,validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]
    
    d_train = lgb.Dataset(X,y)
    d_val = lgb.Dataset(valid_x, valid_y)
    
    
    #run traning
    model = lgb.train(
    params,
    train_set = d_train,
    num_boost_round = 1000,
    valid_sets = d_val,
    feval = f_pr_auc,
    verbose_eval = 20,
    early_stopping_rounds = 3
    )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold,1,0)
    
    #cal score 
    recall = recall_score(valid_y , valid_pred)
    precision = precision_score(valid_y , valid_pred)
    auc_score = roc_auc_score(valid_y , valid_prob)
    
    #append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)
    print('============================================================')

Training until validation scores don't improve for 3 rounds
[20]	valid_0's auc: 0.797673	valid_0's pr+auc: 0.792522
Early stopping, best iteration is:
[25]	valid_0's auc: 0.801171	valid_0's pr+auc: 0.797338
Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[2]	valid_0's auc: 0.788479	valid_0's pr+auc: 0.318902
Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[2]	valid_0's auc: 0.758498	valid_0's pr+auc: 0.358092
Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[10]	valid_0's auc: 0.807468	valid_0's pr+auc: 0.551358
Training until validation scores don't improve for 3 rounds
Early stopping, best iteration is:
[14]	valid_0's auc: 0.802301	valid_0's pr+auc: 0.974445


In [132]:
print(np.mean(auc_scores))

0.7915834292563759


In [133]:
test_err = pd.read_csv('data/test_err_data.csv')
display(test_err.head())

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,20201101030227,model_1,04.16.3553,31,1
1,30000,20201101030227,model_1,04.16.3553,33,2
2,30000,20201101030228,model_1,04.16.3553,15,1
3,30000,20201101030256,model_1,04.16.3553,22,1
4,30000,20201101030300,model_1,04.16.3553,11,1


In [134]:
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [135]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number, 42))

for preson_idx, err in tqdm(id_error):
    test_x[person_idx - test_user_id_min, err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
print(test_x.shape)

100%|██████████| 16532648/16532648 [00:34<00:00, 475101.04it/s]

(14999, 42)


In [136]:
models

In [142]:
pred_y_list = []
for model in models:
    pred_y = model.predict(test_x)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [143]:
pred_ensemble

array([[0.20553663],
       [0.20553663],
       [0.20553663],
       ...,
       [0.20553663],
       [0.20553663],
       [0.20553663]])

In [144]:
sample_submission = pd.read_csv('data/sample_submission.csv')

In [145]:
sample_submission

,user_id,problem
0,30000,0
1,30001,0
2,30002,0
3,30003,0
4,30004,0
...,...,...
14994,44994,0
14995,44995,0
14996,44996,0
14997,44997,0


In [146]:
sample_submission['problem'] = pred_ensemble.reshape(-1)

In [147]:
sample_submission

,user_id,problem
0,30000,0.205537
1,30001,0.205537
2,30002,0.205537
3,30003,0.205537
4,30004,0.205537
...,...,...
14994,44994,0.205537
14995,44995,0.205537
14996,44996,0.205537
14997,44997,0.205537


In [79]:
sample_submission.to_csv("result/dacon_basline.csv", index= False)
sample_submission

,user_id,problem
0,30000,0.205537
1,30001,0.205537
2,30002,0.205537
3,30003,0.205537
4,30004,0.205537
...,...,...
14994,44994,0.205537
14995,44995,0.205537
14996,44996,0.205537
14997,44997,0.205537
